In [ ]:
from keras.activations import silu, swish, linear, tanh, leaky_relu, sigmoid, relu, elu, selu, gelu, linear
from keras.layers import Input, LSTM, Dense, GaussianNoise, BatchNormalization, Activation, PReLU,Dropout
from keras.initializers.initializers_v2 import RandomNormal, RandomUniform
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as ex
import plotly.io as pio
from keras.regularizers import l1, l2, L1L2
from keras.optimizers import nadam_v2, rmsprop_v2, adamax_v2
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
from keras.models import Sequential
import onnx
import tf2onnx
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA,KernelPCA, IncrementalPCA, SparsePCA
from joblib import dump, load
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from datetime import datetime
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import glob
import os
from scipy.signal import savgol_filter
import numpy as np
import seaborn as sn
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
model_version = 'v4.0.0'

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
# Directories for expreiments .csvs
source_dir = 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/smoothed/With_cooling_feature_for_Model_training_5s/'
testdata_dir = 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/smoothed/With_cooling_feature_for_Model_training_5s/Testing_data/'
preprocessor_dir = 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/AI_Model/preprocessor/'
model_dir = 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/AI_Model/'
log_dir = "C:/Temperaturkompensation/logs/" + \
    datetime.now().strftime("%Y%m%d-%H%M%S")
weights_path = str(model_dir+"/weights3/weights")
image_path = 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\csvs\\smoothed\\AI_Model\\Backup\\Z_Welle\\' + model_version + '\\'


In [ ]:
from keras.losses import LossFunctionWrapper, mean_absolute_error, mean_squared_error
import tensorflow as tf
import keras.backend as k


def max_loss(y_true: tf.Tensor, y_pred: tf.Tensor):
    error = tf.multiply(tf.add(tf.multiply(tf.reduce_max(
        tf.abs((y_true - y_pred))), 1), mean_absolute_error(y_true, y_pred)), 1)
    return error


def max_loss_v1(y_true: tf.Tensor, y_pred: tf.Tensor):
    error = tf.add(tf.multiply(tf.reduce_max(tf.abs((y_true - y_pred))), 1), tf.sqrt(mean_squared_error(y_true, y_pred)))
    return error

def max_loss_v2(y_true: tf.Tensor, y_pred: tf.Tensor):
    error = tf.add(tf.multiply(tf.pow(tf.reduce_max(tf.abs((y_true - y_pred))),2), 1), mean_squared_error(y_true, y_pred))
    return error

def max_loss_v3_mae_std(y_true: tf.Tensor, y_pred: tf.Tensor):
    mean_ae= mean_absolute_error(y_true, y_pred)
    vars= tf.pow(tf.subtract(tf.abs(y_true - y_pred),mean_ae),2)
    errors_std= tf.sqrt(tf.reduce_mean(vars))
    #errors_std= tf.reduce_mean(vars)
    #error = tf.add(mean_squared_error(y_true, y_pred), errors_std)
    error = tf.add(mean_ae, errors_std)
    return error

def calc_fidelity(y_true: tf.Tensor, y_pred: tf.Tensor):
    # session= k.get_session()
    # y_true_array= session.run(y_true)
    # y_pred_array= session.run(y_pred)
    # cc= np.corrcoef(y_true_array,y_pred_array)
    NRMSE= tf.divide(tf.sqrt(mean_squared_error(y_true, y_pred)),tf.subtract(tf.reduce_max(y_true),tf.reduce_min(y_true)))
    fidelity= tf.abs(tf.multiply(100.0, tf.subtract(1.0,NRMSE)))
    return fidelity
    
special_loss = LossFunctionWrapper(max_loss)
fidelity_wrapper= LossFunctionWrapper(calc_fidelity)


In [ ]:
# Implement special preprocessing steps
# 1 calculate new Feature
# 2 make rounding


from sklearn.preprocessing import FunctionTransformer


def special_preprocess(data: np.ndarray):
    delta_t_spindel_bett = data[:, 1] - data[:, 0]
    delta_t_spindel_bett = delta_t_spindel_bett.reshape((-1, 1))
    result = np.concatenate((data[:, 2:5], delta_t_spindel_bett), axis=1)
    return result


def rounding(signals: np.ndarray):
    return np.round(signals, 2)


def remove_t_bett(data: np.ndarray):
    result = data[:, 1:]
    result[:, 0] = result[:, 0] - data[:, 0]
    result[:, 1] = result[:, 1] - data[:, 0]
    return result

def remove_t_bett_v2(data: np.ndarray): # 1 X window X features --> 1 X [window X (features-1)]
    result= data[:,:,1:].copy()
    result[:,:, 0] = result[:,:, 0] - data[:,:, 0]
    result[:,:, 1] = result[:,:, 1] - data[:,:, 0]
    result= np.reshape(result,newshape=(data.shape[0],-1))
    #result= result[:,:-1]
    return result   

data = np.asarray([[[40, 90, 300], [500, 1500, 5000]],
                   [[60, 90, 300], [500, 2000, 6000]],
                   [[80, 90, 300], [500, 3000, 7000]],
                   [[100, 90, 300], [500, 4000, 8000]]])
print(data.shape)
new_data= remove_t_bett_v2(data)
print(new_data.shape)
print(new_data)

window = 120
shift = 1

In [ ]:
# selected_Columns= ['diff', 'drhz','t_bett',]#,'t_kluelung']# , 'diff'
# [ 't_bett','t_spindel', 't_y_achse', 't_z_achse', 'drhz', 't_motor','t_kss_tank', 't_raum', 't_spindel_vor', 't_spindel_rueck','t_schwenkantrieb']

output_variable = 'welle_z_iterpolated'#'welle_z_iterpolated'#'welle_z_upsampled'#'z_welle_ok'  #'z_tcp_ok'# 'welle_z'#''#'smoothed_z_welle_ok'#
output_variable_smoothed= output_variable +'_smoothed' 
scaler = StandardScaler(with_mean=True, with_std=True)
minmax = MinMaxScaler(feature_range=(-1, 1))
post_processer = StandardScaler(with_mean=True, with_std=True)#MinMaxScaler(feature_range=(-1, 1))
pca = PCA(n_components=6)
#pca = IncrementalPCA(n_components=10)
# Data Parameters
selected_Columns = ['t_bett', 't_motor', 't_spindle', 'M8','M121', 'M127', 'M7']#['t_bett', 't_motor', 't_spindel', 'M8','M121', 'M127', 'M7']
#,('smoothing_data', FunctionTransformer(smoothing_data))('remove_t_bett', FunctionTransformer(remove_t_bett_v2)),
proposed_pipline = Pipeline(steps=[ ('remove_t_bett', FunctionTransformer(remove_t_bett_v2)),('rounding', FunctionTransformer(rounding)), ('stdscaler', scaler), ('pca', pca),('std2',StandardScaler(with_mean=True, with_std=True))])  # ('std2',StandardScaler(with_mean=True, with_std=True))Pipeline(steps=[('rounding', FunctionTransformer(rounding))])#('pca', pca)
preprocessor_name = "preprocessor.p"
smoothed_output= False
reset = True

In [ ]:
# def generateDataSource(signal_input=None, output_length: int = 1, signal_output=None, window=1, shift=1):
#     Signal_Length = signal_input.shape[0]
#     num_samples = int((Signal_Length - (window-1) * shift))
#     x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
#     y = np.zeros(shape=(num_samples, output_length, 1))
#     for i in range(num_samples):
#         x[i] = np.asarray([signal_input[i + j * shift] for j in range(0,window)])
#         y[i] = signal_output[i + (window-1) * shift :i+ (window-1) * shift + output_length]
#     return x, y

In [ ]:
def smoothing_data(input: pd.DataFrame, window= 240):
    shift_value= int(-window/2)
    data= input.copy()
    # data['t_bett_smoothed']= data['t_bett'].rolling(window= window).mean().shift(shift_value)
    # data['t_motor_smoothed']= data['t_motor'].rolling(window= window).mean().shift(shift_value)
    # data['t_spindle_smoothed']= data['t_spindle'].rolling(window= window).mean().shift(shift_value)
    # data[str(output_variable+'_smoothed')]= data[output_variable].rolling(window= window).mean().shift(shift_value)
    data['t_bett']= data['t_bett'].rolling(window= window).mean().shift(shift_value)
    data['t_motor']= data['t_motor'].rolling(window= window).mean().shift(shift_value)
    data['t_spindle']= data['t_spindle'].rolling(window= window).mean().shift(shift_value)
    data[str(output_variable+'_smoothed')]= data[output_variable].rolling(window= window).mean().shift(shift_value)
    data= data[ window+1:-window]###Remove nan
    return data

In [ ]:
def smoothing_data___(input: pd.DataFrame, window= 240):
    return input

In [ ]:
def generateDataSource_v0(signal_input=None, output_length: int = 1, signal_output=None, window=1, shift=1): ## For flatten version
    Signal_Length = signal_input.shape[0]
    features_count= signal_input.shape[1]
    num_samples = int((Signal_Length - (window-1) * shift))
    #x = np.zeros(shape=(num_samples, window*(features_count + 1) - 1))
    x = np.zeros(shape=(num_samples, window*(features_count)))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i]= np.asarray([signal_input[i + j * shift] for j in range(0,window)]).flatten()
        # x[i] = np.concatenate((np.asarray([signal_input[i + j * shift] for j in range(0,window)]).flatten(),
        #                         (np.asarray([signal_output[i + j * shift] for j in range(0,window-1)]).flatten())))
        y[i] = signal_output[i + (window-1) * shift :i+ (window-1) * shift + output_length]
    return x, y

In [ ]:
def generateDataSource(signal_input=None, output_length: int = 1, signal_output=None, window=1, shift=1):## for3d array without label
    Signal_Length = signal_input.shape[0]
    features_count= signal_input.shape[1]
    num_samples = int((Signal_Length - (window-1) * shift))
    x = np.zeros(shape=(num_samples,window, features_count))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i]= np.asarray([signal_input[i + j * shift] for j in range(0,window)])
        # x[i] = np.concatenate((np.asarray([signal_input[i + j * shift] for j in range(0,window)]).flatten(),
        #                         (np.asarray([signal_output[i + j * shift] for j in range(0,window-1)]).flatten())))
        y[i] = signal_output[i + (window-1) * shift :i+ (window-1) * shift + output_length]
    return x, y

In [ ]:
def generateDataSource__(signal_input=None,output_length: int = 1, signal_output=None, window=0, shift=0):## for3d array without label
    x_train_windowed= TimeseriesGenerator(signal_input,targets= signal_output, length=window, sampling_rate= shift, batch_size= 1)
    print('.... ' , x_train_windowed[0])
    x= np.asarray([x_train_windowed[i][0].flatten() for i in range(0, len(x_train_windowed))])
    x= x[1:]
    y= np.asarray([x_train_windowed[i][1] for i in range(0, len(x_train_windowed))])
    y= y[:-1]
    return x, y

In [ ]:
def generateDataSource_v3(signal_input=None, output_length: int = 1, signal_output=None, window=1, shift=1):## for3d array with label
    Signal_Length = signal_input.shape[0]
    features_count= signal_input.shape[1]
    num_samples = int((Signal_Length - (window-1) * shift))
    #x = np.zeros(shape=(num_samples, window*(features_count + 1) - 1))
    x = np.zeros(shape=(num_samples,window, features_count + 1))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i]= np.asarray([np.append(signal_input[i + j * shift],signal_output[i + j * shift]) if j < (window-1) else
                           np.append(signal_input[i + j * shift],np.asanyarray([0])) for j in range(0,window)])
        y[i] = signal_output[i + (window-1) * shift :i+ (window-1) * shift + output_length]
    return x, y

In [ ]:
x= np.asarray([
    [10,1,1,1,1],
    [20,2,2,2,2],
    [30,3,3,3,3],
    [40,4,4,4,4],
    [50,5,5,5,5],
    [60,6,6,6,6],
    [70,7,7,7,7],
    [80,8,8,8,8],
    [90,9,9,9,9],
    [100,10,10,10,10]
])
y= np.asanyarray([11,22,33,44,55,66,77,88,99,1010])
dx, dy= generateDataSource(signal_input=x, signal_output=y,window=2,shift=3)
print(dx)
print (dy)
#dx_new= remove_t_bett_v2(dx)
#dx_new

Load Training Data

In [ ]:
# Extract Data source from all signals and create Training dataset
working_dir = source_dir
os.chdir(working_dir)
x_train: np.ndarray = None
y_train: np.ndarray = None
#selected_Columns= ['t_motor','t_spindel', 'drhz']
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df = pd.read_csv(file)
    if df.isnull().values.any():
        print ('with nan')
    prev_time=None
    drop_list= []
    signals_filtered= smoothing_data(df, window= 24)
    if signals_filtered.isnull().values.any():
        print ('with nan')
    input_signal= signals_filtered[selected_Columns].to_numpy()
    output_signal= 1000 * signals_filtered[output_variable_smoothed].to_numpy()
    windowed_data_x, windowed_data_y= generateDataSource(signal_input= input_signal, output_length=1, signal_output=output_signal, window= window, shift=shift)
    #print (windowed_data_x[:3])
    #print (windowed_data_y[:3])
    if x_train is None:
        x_train = windowed_data_x
        y_train= windowed_data_y
    else:
        x_train = np.concatenate((x_train, windowed_data_x))
        y_train = np.concatenate((y_train, windowed_data_y))
    print('input: ', x_train.shape, ' output:', y_train.shape)
    print('Input Nans: ',np.isnan(x_train).any())
    print('Input Nans: ',np.isnan(y_train).any())
    print('###########################')

Load Testing Data

In [ ]:
# Extract Data source from all signals and create Training dataset
working_dir = testdata_dir
os.chdir(working_dir)
x_test: np.ndarray = None
y_test: np.ndarray = None
#selected_Columns= ['t_motor','t_spindel', 'drhz']
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df = pd.read_csv(file)
    if df.isnull().values.any():
        print ('with nan')
    #signals = signals[selected_Columns]
    signals_filtered= smoothing_data(df, window= 24)
    if signals_filtered.isnull().values.any():
        print ('with nan')
        exit()
    input_signal= signals_filtered[selected_Columns].to_numpy()
    output_signal= 1000 * signals_filtered[output_variable_smoothed].to_numpy()
    windowed_data_x, windowed_data_y= generateDataSource(signal_input= input_signal, output_length=1, signal_output=output_signal, window= window, shift=shift)
    if x_test is None:
        x_test = windowed_data_x
        y_test= windowed_data_y
    else:
        x_test = np.concatenate((x_test, windowed_data_x))
        y_test = np.concatenate((y_test, windowed_data_y))
    print('input: ', x_test.shape, ' output:', y_test.shape)
    print('Input Nans: ',np.isnan(x_test).any())
    print('Input Nans: ',np.isnan(y_test).any())
    print('###########################')

In [ ]:
y_train= y_train.reshape((-1,1))
y_test= y_test.reshape((-1,1))

Create preprocessor

In [ ]:
pipeline= None
print('input: ', x_train.shape, ' output:', y_train.shape)
if pipeline is None:
    try:
        if not reset:
            pipeline = load(str(preprocessor_dir + preprocessor_name))
            print('preprocessor is loaded successfully')
        else:
            pipeline = proposed_pipline
            pipeline = pipeline.fit(x_train)
            dump(pipeline, str(preprocessor_dir + preprocessor_name))
            print('preprocessor is saved successfully')
            print('explained_variance_ratio_= ',
                 pipeline.steps[3][1].explained_variance_ratio_)
    except FileNotFoundError as err:
        pipeline = proposed_pipline
        pipeline = pipeline.fit(x_train)
        dump(pipeline, str(preprocessor_dir + preprocessor_name))
        # initial_type = [('float_input', FloatTensorType(
        #     [None,  x_train.shape[1]]))]
        # onx = convert_sklearn(pipeline, initial_types=initial_type)
        # with open(str(preprocessor_dir + "preprocessor.onnx"), "wb") as f:
        #     f.write(onx.SerializeToString())
        print('preprocessor is saved successfully')

Create Postprocessor

In [ ]:
post_processer= post_processer.fit(y_train)

Apply Postprocessor

In [ ]:
y_train_post= y_train#post_processer.transform(y_train)
y_test_post= y_test#post_processer.transform(y_test)

Apply preprocessing on the data

In [ ]:
x_train_preprocessed= pipeline.transform(x_train)#[:,[0,4,5]]
x_test_preprocessed= pipeline.transform(x_test)#[:,[0,4,5]]

In [ ]:
x_train_preprocessed.shape

In [ ]:
y_train_post.shape

In [ ]:
df= pd.DataFrame(x_train_preprocessed)#(np.concatenate([x_train_preprocessed,x_test_preprocessed]))
df['output']= y_train_post.flatten()#np.concatenate([y_train_post.flatten(),y_test_post.flatten()])
# df= pd.DataFrame(x_test_preprocessed)
# df['output']= y_test.flatten()
# df.columns


In [ ]:
df.hist(figsize=(25,25))

In [ ]:
correlations = df.corr().round(4)

fig, ax = plt.subplots(figsize=(30, 15))
plt.title('Heat-Map für die Korrelationsmatrix')
sn.heatmap(correlations, annot=True, vmin=-1, vmax=1, cmap='magma', annot_kws={"size": 26, 'color': 'black'})
plt.show()

In [ ]:
print(correlations['output'])

In [ ]:
x_test_preprocessed.shape

In [ ]:
def build_Model(X_train, y_train):
    activation_Function = leaky_relu
    dropout = 0.01
    l1_v = 0.0008
    L2_v = 0.0008
    hidden_layers_structure = [500,400,300,200,100,50,20]
    unroll = False
    kernal_init = RandomUniform()  # 'he_normal'
    optimizer = nadam_v2.Nadam()
    #optimizer= adamax_v2.Adamax(learning_rate= 0.0001)
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    # model.add(Dense(100, activation=activation_Function))
    # #model.add(Dropout(dropout))
    # model.add(BatchNormalization()), kernel_initializer='he_normal', bias_initializer= 'he_normal'
    for i in range(len(hidden_layers_structure)):
        model.add(Dense(hidden_layers_structure[i], bias_regularizer =L1L2(l1=l1_v, l2=L2_v), kernel_regularizer=L1L2(l1=l1_v, l2=L2_v), kernel_initializer='he_normal', bias_initializer= 'he_normal'))
        #model.add(BatchNormalization())
        model.add(Activation(activation_Function))
        #model.add(PReLU())
        #model.add(BatchNormalization())
        #model.add(Dropout(dropout))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer= 'Adamax', loss=special_loss, metrics=[mape])
    return model

In [ ]:
def map(old_value, old_min, old_max, new_max, new_min):
    new_value= ( (old_value - old_min) / (old_max - old_min) ) * (new_max - new_min) + new_min
    return new_value
def calculate_weights(y_train: np.ndarray, occurance_threshold= 0):

    y_train_rounded = np.round(y_train,decimals=3)
    N = len(y_train_rounded)
    y_train_rounded = np.reshape(y_train_rounded, newshape=(N,))
    weights = np.ones(shape=(N,),dtype= np.float32)
    for i in range(0, N):
        current_value = y_train_rounded[i]
        occurences = np.count_nonzero(y_train_rounded == current_value)
        if occurences <= occurance_threshold:
             weights[i]=  N/(occurences + 1 )#N/(occurences + y_train_rounded[i])#N/occurences # 0
        else:
            weights[i] =  N/(occurences + 1)#(8 if y_train_rounded[i] < 20 else 1)
    old_min= weights.min()
    old_max= weights.max()
    new_min = 1
    new_max = 2
    #weights_scaled= np.apply_along_axis(map, 1, weights)
    weights_scaled= np.asanyarray([ map(weights[i], old_min, old_max, new_max, new_min) for i in range(0, weights.shape[0])]).reshape(weights.shape)
    #weights = weights - weights.min() + 1
    #weights = (weights - weights.min())/(weights.max() - weights.min())
    fig= make_subplots(rows=2,cols=1,shared_xaxes= True, print_grid= True,  vertical_spacing=0.02)
    fig.add_trace(go.Line(y=weights_scaled,name='weight of Labels')#visualisation_selected_Columns[-2])
    , row= 1, col= 1)
    fig.add_trace(go.Line(y=np.reshape(y_train, newshape=(N,)),name='Labels')#visualisation_selected_Columns[-2])
    , row= 2, col= 1)
    # fig.update_yaxes(title_text= 'weight', row= 1, col= 1)
    # fig.update_xaxes(title_text= 'label value', row= 1, col= 1)
    fig.update_layout(height=900, width=900, title_text= 'weight of Labels')
    fig.show()

    return weights_scaled

Training the model

In [ ]:
# Training Block
from keras.callbacks import LambdaCallback
checkpoint = ModelCheckpoint(filepath=str(weights_path+'best_model.h5'), monitor='val_loss', verbose=1,
                             save_best_only=True, save_weights_only=True, mode='min', save_freq='epoch')
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[0].get_weights()))
earlyStopping = EarlyStopping(
    monitor='val_loss', mode='min', patience=400, verbose=1)
model = build_Model(X_train=x_train_preprocessed, y_train=y_train_post)
weights = calculate_weights(y_train=y_train_post, occurance_threshold= 5)
reset= True
try:
    if not reset:
        model.load_weights(filepath=weights_path)
        print('Previous weights loaded Successfully')
except:
    print('No Previous weights')
print('input shape ', model.input_shape)
print(model.output_shape)
print(model.summary())
summary = model.fit(x=x_train_preprocessed, y=y_train_post, shuffle=True, batch_size=64,  epochs=2000,   validation_split= 0.1,#validation_data=(x_test_preprocessed, y_test_post),# 
                    callbacks=[earlyStopping], verbose=2, workers=64, use_multiprocessing=True, sample_weight=weights)
model.save(str(model_dir+'model.h5'), save_format='h5')
input_signature = [tf.TensorSpec(
    [None, x_train_preprocessed.shape[1]], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(
    model=model, input_signature=input_signature, opset=10)
onnx.save_model(onnx_model, str(model_dir+'model.onnx'))
print(".onnx model saved successfully")

In [ ]:
# model.save_weights(weights_path,save_format='h5')
# model.save_weights(weights_path,save_format='tf'),30
# model.save(str(model_dir+'Temp_model.h5'), save_format='h5',include_optimizer=True,save_traces=True, overwrite=True)

In [ ]:
loss = summary.history['loss']
val_loss = summary.history['val_loss']
fig = make_subplots(rows=1, cols=1, shared_xaxes=True,
                    print_grid=True,  vertical_spacing=0.02)
fig.add_trace(go.Line(y=loss, name='Training Loss'), row=1, col=1)
fig.add_trace(go.Line(y=val_loss, name='Validation Loss'), row=1, col=1)
fig.update_xaxes(title_text='Epochs', row=1, col=1)
fig.update_yaxes(title_text='Max_Loss', row=1, col=1)
fig.update_layout(height=900, width=900, title_text='training_curve')
fig.show()
pio.write_image(fig, str(image_path+'training_curve.png'), format='png')

In [ ]:
import numpy as np
tolerance = 5
x_steps = np.linspace(0, 50, 10)
lowerline = x_steps - tolerance * np.sin(np.pi/4)
upperline = x_steps + tolerance * np.sin(np.pi/4)
# Create Tolerance Lines
tolerance = 10
x_steps = np.linspace(0, 50, 10)
lowerline_actual = x_steps - tolerance * np.sin(np.pi/4)
upperline_actual = x_steps + tolerance * np.sin(np.pi/4)

In [ ]:
import numpy as np
from keras.models import load_model
#from keras.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
# model= build_Model(X_train= results,y_train= targets)
# model.load_weights(filepath= weights_path)
# model.save(str(model_dir+'model.h5'), save_format='h5')
model = Sequential()
model = load_model(filepath=str(model_dir+'model.h5'), compile=False)
pipeline = load(str(preprocessor_dir+preprocessor_name))
print(model.summary())
print(model.input_shape)
# pred_train= model.predict(x_train)
# print ('mse= ',np.mean(mean_squared_error(y_train,pred_train)), ' mae= ', np.mean(mean_absolute_error(y_train,pred_train)), ' mape= ', np.mean(mean_absolute_percentage_error(y_train,pred_train)))
# pred_test= model.predict(x_test)
# print ('mse= ',np.mean(mean_squared_error(y_test,pred_test)), ' mae= ', np.mean(mean_absolute_error(y_test,pred_test)), ' mape= ', np.mean(mean_absolute_percentage_error(y_test,pred_test)))
pred = model.predict(x_train_preprocessed)
#pred= post_processer.inverse_transform(pred)
#print (' For All Data mse= ',mean_squared_error(targets,pred), ' mae= ', mean_absolute_error(targets,pred), ' mape= ', mean_absolute_percentage_error(targets,pred))
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Results of DENSE Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.grid()
plt.plot(x_steps, upperline, label='Upper Bound', color='green')
plt.scatter(y_train_post, pred, label='predictions_train', color='blue')
plt.plot(x_steps, lowerline, label='lower Bound', color='green')
plt.plot(x_steps, x_steps, label='Optimal line')
plt.plot(x_steps, lowerline_actual, label='Actual lower Bound', color='red')
plt.plot(x_steps, upperline_actual, label='Actual Upper Bound', color='red')
plt.legend()
plt.savefig(str(image_path+'training_results.png'))
plt.show()

In [ ]:
import numpy as np
from keras.models import load_model
#from keras.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
# model= build_Model(X_train= results,y_train= targets)
# model.load_weights(filepath= weights_path)
# model.save(str(model_dir+'model.h5'), save_format='h5')
model = Sequential()
model = load_model(filepath=str(model_dir+'model.h5'), compile=False)
pipeline = load(str(preprocessor_dir+preprocessor_name))
print(model.summary())
print(model.input_shape)
# pred_train= model.predict(x_train)
# print ('mse= ',np.mean(mean_squared_error(y_train,pred_train)), ' mae= ', np.mean(mean_absolute_error(y_train,pred_train)), ' mape= ', np.mean(mean_absolute_percentage_error(y_train,pred_train)))
# pred_test= model.predict(x_test)
# print ('mse= ',np.mean(mean_squared_error(y_test,pred_test)), ' mae= ', np.mean(mean_absolute_error(y_test,pred_test)), ' mape= ', np.mean(mean_absolute_percentage_error(y_test,pred_test)))
pred = model.predict(x_test_preprocessed)
#pred= post_processer.inverse_transform(pred)
#print (' For All Data mse= ',mean_squared_error(targets,pred), ' mae= ', mean_absolute_error(targets,pred), ' mape= ', mean_absolute_percentage_error(targets,pred))
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Results of DENSE Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.grid()
plt.plot(x_steps, upperline, label='Upper Bound', color='green')
plt.scatter(y_test_post, pred, label='predictions_train', color='blue')
plt.plot(x_steps, lowerline, label='lower Bound', color='green')
plt.plot(x_steps, x_steps, label='Optimal line')
plt.plot(x_steps, lowerline_actual, label='Actual lower Bound', color='red')
plt.plot(x_steps, upperline_actual, label='Actual Upper Bound', color='red')
plt.legend()
plt.savefig(str(image_path+'training_results.png'))
plt.show()

In [ ]:
data_path = str(source_dir + 'validate_Data/')
#image_path= str(source_dir + 'results/')
pipeline = load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
model = Sequential()
model = load_model(str(model_dir+'model.h5'), compile=False)
print('Model is loaded successfully')
os.chdir(data_path)
x_test: np.ndarray = None
y_test: np.ndarray = None
errors = []
error_df = pd.DataFrame()
scatter_mode= 'lines'
#selected_Columns= ['t_motor','t_spindel', 'drhz']
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df = pd.read_csv(file)
    if df.isnull().values.any():
        print ('with nan')
    signals = df
    prev_time=None
    drop_list= []
    print('Before Filtering, ', len(signals))
    for index,row in signals.iterrows():
        current_time= datetime.strptime(row['date'], "%Y-%m-%d %H:%M:%S.%f")
        if prev_time is None:
            prev_time= current_time
        elif (current_time - prev_time).total_seconds() < 5:
            drop_list.append(datetime.strftime(current_time, "%Y-%m-%d %H:%M:%S.%f"))
            df= df[df['date'] != row['date']]
        prev_time= current_time
    signals_filtered= df
    print('After Filtering, ', len(signals_filtered), ', deleted= ', len(drop_list))
    #signals = signals[selected_Columns]
    signals_filtered= smoothing_data(signals_filtered)
    if signals_filtered.isnull().values.any():
        print ('with nan')
    #signals = preprocess_data(signals, kuelung=['M8', 'M121', 'M127', 'M7'])
    input_signal= signals_filtered[selected_Columns].to_numpy()
    output_signal= 1000 * signals_filtered[output_variable].to_numpy()
    windowed_data_x, windowed_data_y= generateDataSource(signal_input= input_signal, output_length=1, signal_output=output_signal, window= window, shift=shift)
    x_data_preprocessed= pipeline.transform(windowed_data_x)
    predictions= model.predict(x_data_preprocessed)
    y_true = windowed_data_y.reshape((-1,))
    predictions = np.reshape(predictions, newshape= y_true.shape)
    diff = np.abs(y_true - predictions)
    mae_v = ' mae= ' + str(np.mean(mean_absolute_error(y_true, predictions)).round(2)) +' Micro_Meter '  
    max_v = ' max= ' + str(np.max(np.abs(y_true - predictions)).round(2)) +' Micro_Meter '  
    ######################
    # Visualization of lf-data with predicted results
    visualisation_df= signals_filtered[window-1:]
    print(visualisation_df.columns)
    visualisation_df['predictions'] = predictions
    visualisation_df[output_variable]= y_true
    fig = make_subplots(rows= 9, cols=1, shared_xaxes=True, print_grid=True,  vertical_spacing=0.02)
    # ##t_bett
    fig.add_trace(go.Line(y=visualisation_df['t_bett'], name='t_bett'), row= 1, col=1)
    fig.update_yaxes(title_text='t_bett C°', row= 1, col=1)
    ###t_motor
    fig.add_trace(go.Line(y=visualisation_df['t_motor'], name='t_motor'), row= 2, col=1)
    fig.update_yaxes(title_text='t_motor C°', row= 2, col=1)
    ###t_spindel
    fig.add_trace(go.Line(y=visualisation_df['t_spindle'], name='t_spindle'), row= 3, col=1)
    fig.update_yaxes(title_text='t_spindel C°', row= 3, col=1)
    ###M8
    fig.add_trace(go.Line(y=visualisation_df['M8'], name='M8'), row= 4, col=1)
    fig.update_yaxes(title_text='M8', row= 4, col=1)
    ###M121
    fig.add_trace(go.Line(y=visualisation_df['M121'], name='M121'), row= 5, col=1)
    fig.update_yaxes(title_text='M121', row= 5, col=1)
    ###M127
    fig.add_trace(go.Line(y=visualisation_df['M127'], name='M127'), row= 6, col=1)
    fig.update_yaxes(title_text='M127', row= 6, col=1)
    ###M7
    fig.add_trace(go.Line(y=visualisation_df['M7'], name='M7'), row= 7, col=1)
    fig.update_yaxes(title_text='M7', row= 7, col=1)
    ###verlagerung
    fig.add_trace(go.Line(y=  visualisation_df[output_variable], name='z_welle_Soll'), row= 8, col=1)
    fig.add_trace(go.Line(y=  visualisation_df['predictions'], name='KI_z_welle_Ist'), row= 8, col=1)
    #fig.add_trace(go.Line(y= 1000* visualisation_df['prediction2'], name='KI_z_welle_during_Manufacturing'), row= 8, col=1)
    fig.update_yaxes(title_text='Verlagerung Mic-Meter', row= 8, col=1)
    ###Fehler
    experiment_errors = (visualisation_df[output_variable] - visualisation_df['predictions']).to_list()
    fig.add_trace(go.Scatter( y=   experiment_errors, name='Prediction Error', mode= scatter_mode),  row= 9 , col= 1)
    ## Draw the tolerence +-5
    fig.add_trace(go.Scatter(y= np.full_like(experiment_errors,5), name='+5 Max Error', mode= scatter_mode),  row= 9 , col= 1)
    fig.add_trace(go.Scatter(y= np.full_like(experiment_errors,-5), name='-5 Min Error', mode= scatter_mode),  row= 9 , col= 1)
    fig.update_yaxes(title_text='Fehler Mic-Meter', row= 9, col=1)
    fig.update_xaxes(title_text='Zeit (1= 2 Minuten)', row=9, col=1)
    fig.update_layout(height=900, width=1200, title_text=str(file + mae_v + max_v))
    fig.show()
    pio.write_image(fig, str(image_path + file+'.png'), format='png')
    current_error_df = pd.DataFrame(
        {'verlagerung': visualisation_df['predictions'].to_list(), 'error': experiment_errors})
    if error_df is None:
        error_df = current_error_df
    else:
        error_df = pd.concat([error_df, current_error_df])
    errors.extend(experiment_errors)
